## Note that this notebook should be run in GCP Colab environment

Runtime requirements:
- min. 750 GB disk space
- min. 500 GB RAM

In [1]:
!pwd

/content


In [2]:
!git clone https://github.com/EMBL-EBI-ABC/PerturbationCatalogue.git

Cloning into 'PerturbationCatalogue'...
remote: Enumerating objects: 6587, done.
remote: Counting objects: 100% (1834/1834), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 6587 (delta 1214), reused 1121 (delta 1053), pack-reused 4753 (from 3)
Receiving objects: 100% (6587/6587), 104.18 MiB | 48.05 MiB/s, done.
Resolving deltas: 100% (4385/4385), done.


In [2]:
%cd PerturbationCatalogue/data_exploration/

/content/PerturbationCatalogue/data_exploration


In [3]:
!git pull
!git switch aleks/curate_nadig_2025

Already up to date.
M	data_exploration/curation_tools/ontologies/gene_ont.parquet
M	data_exploration/local_curation_tools.egg-info/SOURCES.txt
Already on 'aleks/curate_nadig_2025'
Your branch is up to date with 'origin/aleks/curate_nadig_2025'.


In [5]:
!uv pip install -r requirements.txt

Using Python 3.12.12 environment at: /usr
Resolved 181 packages in 2.00s
Prepared 49 packages in 890ms
Uninstalled 3 packages in 126ms
Installed 49 packages in 43ms
 + adjusttext==1.3.0
 + anndata==0.12.2
 + array-api-compat==1.12.0
 + blitzgsea==1.3.54
 + decoupler==2.0.1
 + docrep==0.3.2
 + donfig==0.8.1.post1
 + equinox==0.13.2
 + fast-array-utils==1.3.1
 + gprofiler-official==1.0.0
 + igraph==1.0.0
 + jaxopt==0.8.5
 + jaxtyping==0.3.3
 + lamin-utils==0.15.0
 + legacy-api-wrap==1.5
 + legendkit==0.3.6
 + libchebipy==1.0.10
 + lightning==2.6.0
 + lightning-utilities==0.15.2
 + lineax==0.0.8
 + local-curation-tools==0.1.0 (from file:///content/PerturbationCatalogue/data_exploration)
 + marsilea==0.5.6
 + ml-collections==1.1.0
 + mudata==0.3.2
 + mypy-extensions==1.1.0
 + numcodecs==0.16.5
 + ott-jax==0.6.0
 + pandera==0.27.0
 + pertpy==0.11.4
 + pubchempy==1.0.5
 + pyro-api==0.1.2
 + pyro-ppl==1.9.1
 + pytorch-lightning==2.6.0
 + rapidfuzz==3.14.3
 + scanpy==1.11.1
 - scikit-learn==1.

In [6]:
!gcloud init --project "prj-ext-dev-pertcat-437314" --account "zakirov@ebi.ac.uk"

Welcome! This command will take you through the configuration of gcloud.

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You are signed in as: [zakirov@ebi.ac.uk].

Your current project has been set to: [prj-ext-dev-pertcat-437314].

Do you want to configure a default Compute Region and Zone? (Y/n)?  n

The Google Cloud CLI is configured and ready to use!

* Commands that require authentication will use zakirov@ebi.ac.uk by default
* Commands will reference project `prj-ext-dev-pertcat-437314` by default
Run `gcloud help config` to learn how to change individual settings

This gcloud configuration is called [default]. You can create additional configurations if you work with multiple accounts and/or projects.
Run `gcloud topic configuratio

In [7]:
!mkdir -p Perturbseq/non_curated/h5ad/
!gcloud storage cp gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hek293t.h5ad Perturbseq/non_curated/h5ad/

Copying gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hek293t.h5ad to file://Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad

Average throughput: 463.5MiB/s


# Import

In [4]:
import pandas as pd
import json

from curation_tools.curation_tools import (
    CuratedDataset,
    ObsSchema,
    VarSchema,
    Experiment,
    download_file,
    upload_parquet_to_bq
)

# import logging
# logging.basicConfig(
#     level=logging.DEBUG,
#     format="%(asctime)s %(levelname)s %(name)s: %(message)s",
#     handlers=[
#         logging.FileHandler("curation.log"),
#         logging.StreamHandler(),  # keep console output too
#     ],
#     force=True,
# )

/usr/local/lib/python3.12/dist-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# Pre-process the data

### Load the data and construct the dummy anndata object to explore the dataset offline

In [5]:
# from scipy.sparse import csr_matrix
# import numpy as np
# import scanpy as sc
# import snapatac2 as snap

# backed_data = snap.read('Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad')

# obs = backed_data.obs[:].to_pandas()
# var = backed_data.var[:].to_pandas()

# N1 = obs.shape[0]  # Number of objects (e.g., cells)
# N2 = var.shape[0]  # Number of categories (e.g., genes)
# rows = np.arange(N1)
# cols = (np.floor(np.random.permutation(N1)/float(N1)*N2)).astype(int) # Randomly pick N1 objects and assign to N2 categories in almost equal proportion
# w = np.ones(N1)
# conn_matrix = csr_matrix((w, (rows, cols)), shape=(N1, N2), dtype=int)

# adata = sc.AnnData(X=conn_matrix, obs=obs, var=var)

# # save the non-curated AnnData object
# adata.write_h5ad("Perturbseq/non_curated/h5ad/orion_2025_hek293t_dummy.h5ad")

# Initialise the dataset object

In [6]:
noncurated_path = "Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad"
cur_data = CuratedDataset(
    obs_schema=ObsSchema,
    var_schema=VarSchema,
    exp_metadata_schema=Experiment,
    noncurated_path=noncurated_path
)

cur_data.load_data()

Loading data from Perturbseq/non_curated/h5ad/orion_2025_hek293t.h5ad


In [7]:
cur_data.adata.obs

,sample,num_features,guide_target,gene_target,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,pass_guide_filter
AAACCAAAGAAGTAGC-HEK293T_Batch1,HEK293T_Batch1,2,FAM180A_P1P2-1|FAM180A_P1P2-2,FAM180A,6856,24243.0,647.0,2.668812,True
AAACCAAAGAGTACGG-HEK293T_Batch1,HEK293T_Batch1,2,non-targeting_03016|non-targeting_03214,Non-Targeting,6636,20847.0,886.0,4.250012,True
AAACCAAAGAGTGGGC-HEK293T_Batch1,HEK293T_Batch1,2,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,PCDHGC4,5353,14035.0,610.0,4.346277,True
AAACCAAAGAGTGTAA-HEK293T_Batch1,HEK293T_Batch1,2,USP30_P1P2-1|USP30_P1P2-2,USP30,6358,20939.0,589.0,2.812933,True
AAACCAAAGATACGGG-HEK293T_Batch1,HEK293T_Batch1,2,KLRD1_P1P2-1|KLRD1_P1P2-2,KLRD1,5527,14083.0,476.0,3.379961,True
...,...,...,...,...,...,...,...,...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,HEK293T_Batch223,2,non-targeting_00183|non-targeting_01885,Non-Targeting,8112,29296.0,522.0,1.781813,True
GTTGTCTTCAACAGCC-HEK293T_Batch223,HEK293T_Batch223,2,AKAP13_P1P2-1|AKAP13_P1P2-2,AKAP13,10568,88551.0,3035.0,3.427403,True
GTTGTCTTCAGTACTG-HEK293T_Batch223,HEK293T_Batch223,2,TTF2_P1P2-1|TTF2_P1P2-2,TTF2,10735,81286.0,1808.0,2.224245,True
GTTGTCTTCCGCCGTT-HEK293T_Batch223,HEK293T_Batch223,2,MMP7_P1P2-1|MMP7_P1P2-2,MMP7,9248,40733.0,1767.0,4.338006,True


In [8]:
cur_data.adata.var['gene_symbol'] = cur_data.adata.var.index

In [9]:
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 1)
--------------------------------------------------
                     gene_symbol
DDX11L2                  DDX11L2
MIR1302-2HG          MIR1302-2HG
FAM138A                  FAM138A
ENSG00000290826  ENSG00000290826
OR4F5                      OR4F5
...                          ...
ENSG00000277836  ENSG00000277836
ENSG00000278633  ENSG00000278633
ENSG00000276017  ENSG00000276017
ENSG00000278817  ENSG00000278817
ENSG00000277196  ENSG00000277196

[38606 rows x 1 columns]
--------------------------------------------------


# OBS slot curation

In [10]:
cur_data.adata.obs.isna().sum()

,0
sample,0
num_features,0
guide_target,0
gene_target,0
n_genes_by_counts,0
total_counts,0
total_counts_mt,0
pct_counts_mt,0
pass_guide_filter,0


### Extract perturbation symbols

### Rename `guide_target` to `perturbation_name`

In [11]:
cur_data.rename_columns(slot = 'obs', name_dict = {'guide_target': 'perturbation_name'})

Renamed columns in adata.obs: {'guide_target': 'perturbation_name'}


### Rename `gene_target` to `perturbed_target_symbol`

In [12]:
cur_data.rename_columns(slot = 'obs', name_dict = {'gene_target': 'perturbed_target_symbol'})

Renamed columns in adata.obs: {'gene_target': 'perturbed_target_symbol'}


#### Clean up `perturbed_target_symbol` column

In [13]:
cur_data.replace_entries(
    slot="obs",
    column="perturbed_target_symbol",
    map_dict={
        "Non-Targeting": "control_nontargeting"
    }
)

cur_data.show_unique(slot = 'obs', column = 'perturbed_target_symbol')

Streaming output truncated to the last 5000 lines.
 'RPA4',
 'RPAIN',
 'RPAP1',
 'RPAP2',
 'RPAP3',
 'RPE',
 'RPE65',
 'RPF1',
 'RPF2',
 'RPGR',
 'RPGRIP1',
 'RPGRIP1L',
 'RPH3A',
 'RPH3AL',
 'RPIA',
 'RPL10',
 'RPL10A',
 'RPL10L',
 'RPL11',
 'RPL12',
 'RPL13',
 'RPL13A',
 'RPL14',
 'RPL15',
 'RPL17',
 'RPL18',
 'RPL18A',
 'RPL19',
 'RPL21',
 'RPL22',
 'RPL22L1',
 'RPL23',
 'RPL23A',
 'RPL24',
 'RPL26',
 'RPL26L1',
 'RPL27',
 'RPL27A',
 'RPL28',
 'RPL29',
 'RPL3',
 'RPL30',
 'RPL31',
 'RPL32',
 'RPL34',
 'RPL35',
 'RPL35A',
 'RPL36',
 'RPL36A',
 'RPL36AL',
 'RPL37',
 'RPL37A',
 'RPL38',
 'RPL39',
 'RPL39L',
 'RPL3L',
 'RPL4',
 'RPL41',
 'RPL5',
 'RPL6',
 'RPL7',
 'RPL7A',
 'RPL7L1',
 'RPL8',
 'RPL9',
 'RPLP0',
 'RPLP1',
 'RPLP2',
 'RPN1',
 'RPN2',
 'RPP14',
 'RPP21',
 'RPP25',
 'RPP25L',
 'RPP30',
 'RPP40',
 'RPRD1A',
 'RPRD1B',
 'RPRD2',
 'RPRM',
 'RPRML',
 'RPS10',
 'RPS11',
 'RPS12',
 'RPS13',
 'RPS14',
 'RPS15',
 'RPS15A',
 'RPS16',
 'RPS17',
 'RPS18',
 'RPS19',
 'RPS19BP1',
 'RPS2

### Standardise perturbation targets

In [14]:
cur_data.standardize_genes(
    slot='obs',
    input_column='perturbed_target_symbol',
    input_column_type='gene_symbol',
    multiple_entries=False
)

Mapping gene symbols: 100%|████████████████████████████████| 18312/18312 [00:00<00:00, 24581.72it/s]


--------------------------------------------------
Successfully mapped 18306 out of 18312 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['control_nontargeting', 'LINC02911', 'PRY2', 'SMIM2', 'PRY', 'LINC02913']
--------------------------------------------------


### Add `perturbed_target_number` column

In [19]:
cur_data.adata.obs = cur_data.adata.obs.iloc[:,[0,1,2,3,4,5,6,7,8,9,11,12,13]]

In [20]:
cur_data.count_entries(
    slot='obs',
    input_column='perturbed_target_symbol',
    count_column_name='perturbed_target_number',
    sep='|'
)

Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number


### Encode chromosomes as integers

In [21]:
cur_data.chromosome_encoding()

Chromosome encoding applied to perturbed_target_chromosome in adata.obs and stored as 'perturbed_target_chromosome_encoding'.


In [22]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_chromosome_encoding'])

Observation data:
DataFrame shape: (4534299, 2)
--------------------------------------------------
                                                         perturbation_name  \
index                                                                        
AAACCAAAGAAGTAGC-HEK293T_Batch1              FAM180A_P1P2-1|FAM180A_P1P2-2   
AAACCAAAGAGTACGG-HEK293T_Batch1    non-targeting_03016|non-targeting_03214   
AAACCAAAGAGTGGGC-HEK293T_Batch1              PCDHGC4_P1P2-1|PCDHGC4_P1P2-2   
AAACCAAAGAGTGTAA-HEK293T_Batch1                  USP30_P1P2-1|USP30_P1P2-2   
AAACCAAAGATACGGG-HEK293T_Batch1                  KLRD1_P1P2-1|KLRD1_P1P2-2   
...                                                                    ...   
GTTGTCCGTGGGATGG-HEK293T_Batch223  non-targeting_00183|non-targeting_01885   
GTTGTCTTCAACAGCC-HEK293T_Batch223              AKAP13_P1P2-1|AKAP13_P1P2-2   
GTTGTCTTCAGTACTG-HEK293T_Batch223                  TTF2_P1P2-1|TTF2_P1P2-2   
GTTGTCTTCCGCCGTT-HEK293T_Batch223          

In [23]:
cur_data.adata.obs.head()

,sample,total_counts_mt,pass_guide_filter,n_genes_by_counts,num_features,total_counts,perturbation_name,pct_counts_mt,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_number,perturbed_target_chromosome_encoding
index,,,,,,,,,,,,,,,
AAACCAAAGAAGTAGC-HEK293T_Batch1,HEK293T_Batch1,647.0,True,6856,2,24243.0,FAM180A_P1P2-1|FAM180A_P1P2-2,2.668812,ENSG00000189320,FAM180A,protein_coding,chr7:135728348-135748813;-1,7,1,7
AAACCAAAGAGTACGG-HEK293T_Batch1,HEK293T_Batch1,886.0,True,6636,2,20847.0,non-targeting_03016|non-targeting_03214,4.250012,None,control_nontargeting,None,None,None,1,0
AAACCAAAGAGTGGGC-HEK293T_Batch1,HEK293T_Batch1,610.0,True,5353,2,14035.0,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,4.346277,ENSG00000242419,PCDHGC4,protein_coding,chr5:141484997-141512975;1,5,1,5
AAACCAAAGAGTGTAA-HEK293T_Batch1,HEK293T_Batch1,589.0,True,6358,2,20939.0,USP30_P1P2-1|USP30_P1P2-2,2.812933,ENSG00000135093,USP30,protein_coding,chr12:109023089-109088023;1,12,1,12
AAACCAAAGATACGGG-HEK293T_Batch1,HEK293T_Batch1,476.0,True,5527,2,14083.0,KLRD1_P1P2-1|KLRD1_P1P2-2,3.379961,ENSG00000134539,KLRD1,protein_coding,chr12:10226058-10329608;1,12,1,12


### Add guide RNA information

The original pre-print does not contain information about guide RNAs in the `obs` slot.

However, the pre-print references the source of the library - [Maximizing CRISPRi efficacy and accessibility with dual-sgRNA libraries and optimal effectors. Replogle et al 2022](https://doi.org/10.7554/eLife.81856).

Supplementary table s4 contains the information about guide RNAs used in the experiment:

Supplementary table S4: https://elifesciences.org/download/aHR0cHM6Ly9jZG4uZWxpZmVzY2llbmNlcy5vcmcvYXJ0aWNsZXMvODE4NTYvZWxpZmUtODE4NTYtc3VwcDQtdjIueGxzeA--/elife-81856-supp4-v2.xlsx?_hash=AmEXp%2BOl1z1Y4JCBAaVhST5ImQmKAGdfEjGhuJbSJ9Q%3D

In [24]:
cur_data.gene_ont[["synonym", "gene_symbol"]].drop_duplicates()

,synonym,gene_symbol
0,None,None
1,LSAMP,LSAMP
2,KNL1,KNL1
5,LINC02000,LINC02000
7,KRT18P53,KRT18P53
...,...,...
1039922,control_gsh,control_gsh
1039923,control_genedesert,control_genedesert
1039924,control_positive,control_positive
1039925,control_guideonly,control_guideonly


In [25]:
# get synonyms from the gene ontology
syn_df = cur_data.gene_ont[["synonym", "gene_symbol"]].drop_duplicates()
syn_df["gene_symbol"] = syn_df["gene_symbol"].fillna("")
syn_df["synonym"] = syn_df["synonym"].fillna("")
syn_df["all_genes"] = (
    (syn_df["gene_symbol"] + "|" + syn_df["synonym"]).str.lstrip("|").str.rstrip("|")
)
syn_df["all_genes_match"] = syn_df["all_genes"].str.split("|")
syn_df = syn_df.explode("all_genes_match")
syn_df["all_genes_match"] = syn_df["all_genes_match"].str.upper()

# read data from supplementary files
guide_ids_df = pd.read_csv(
    "Perturbseq/supplementary/orion_2025_weissman_dual_crispri_sgrnas.csv"
)
guide_ids_df["gene"] = guide_ids_df["gene"].str.upper()

guide_ids_df = guide_ids_df.merge(
    syn_df, left_on="gene", right_on="all_genes_match", how="left"
)

guide_ids_df["all_genes"] = guide_ids_df["all_genes"].str.split("|")
guide_ids_df = guide_ids_df.explode("all_genes").drop_duplicates()

# create a perturbation name column
guide_ids_df["perturbation_name"] = (
    guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
    + "|"
    + guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
)

guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.upper()

# replace commas with underscores in the perturbation name
guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.replace(
    ",", "_"
)

# create a dual guide RNA sequence column
guide_ids_df["dual_guide_sequence"] = (
    guide_ids_df["protospacer_A"] + "|" + guide_ids_df["protospacer_B"]
).str.upper()

guide_ids_df.loc[
    guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "perturbation_name"
] = (
    guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
    + "|"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
)

# replace NEGATIVE_CONTROL with actual non targeting guide names
guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "perturbation_name"] = (
    guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "sgID_AB"].str.upper()
)

# create dictionaries for guide sequences mapping
guide_ids_dict = (
    guide_ids_df[["perturbation_name", "dual_guide_sequence"]]
    .dropna()
    .drop_duplicates()
    .set_index("perturbation_name")["dual_guide_sequence"]
    .to_dict()
)

# add guide sequences to the obs dataframe
obj_df = cur_data.adata.obs.copy()

# remove the numeric suffices from the perturbation names
obj_df["perturbation_name_matching"] = obj_df["perturbation_name"].str.replace(
    r"-\d(?=\||$)", "", regex=True
)
obj_df["perturbation_name_matching"] = obj_df["perturbation_name_matching"].str.upper()

# map the guide sequences to the perturbation names
obj_df["guide_sequence"] = obj_df["perturbation_name_matching"].map(guide_ids_dict)

obj_df = obj_df[["guide_sequence"]]

obj_df

,guide_sequence
index,
AAACCAAAGAAGTAGC-HEK293T_Batch1,GGAGAGCGTCAAATGAAGAG|GGAGGCTGAAGGCTGCGTCC
AAACCAAAGAGTACGG-HEK293T_Batch1,GGAGGCTATCTGCTAGACTT|GTCCACATGCGGATAGACAC
AAACCAAAGAGTGGGC-HEK293T_Batch1,GGGCCGAACCGCGCCAGAGC|GCGACCGAGAGGCACTGTGT
AAACCAAAGAGTGTAA-HEK293T_Batch1,GTGCCGAAGAGGCCGGGACC|GACGACGGTTCCCGAGACAG
AAACCAAAGATACGGG-HEK293T_Batch1,GATGTTCTTGAATCATCTAT|GGGTTCCTGGAACACTTCAG
...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,GGGGGAATCTGCAAGACCAT|GAGAAGGAGCGAGGCTAAAG
GTTGTCTTCAACAGCC-HEK293T_Batch223,GGAGCACAGGCGCTTCACCG|GGGCAGTCTCGGCAGAGCAC
GTTGTCTTCAGTACTG-HEK293T_Batch223,GAAGTTAGGTGTCCAGAGCA|GAGGTGTCCAGAGCACGGTA


In [26]:
# merge guide sequences with main
cur_data.adata.obs = cur_data.adata.obs.merge(
    obj_df,
    left_index=True,
    right_index=True,
    how="left",
)

cur_data.adata.obs

,sample,total_counts_mt,pass_guide_filter,n_genes_by_counts,num_features,total_counts,perturbation_name,pct_counts_mt,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_number,perturbed_target_chromosome_encoding,guide_sequence
index,,,,,,,,,,,,,,,,
AAACCAAAGAAGTAGC-HEK293T_Batch1,HEK293T_Batch1,647.0,True,6856,2,24243.0,FAM180A_P1P2-1|FAM180A_P1P2-2,2.668812,ENSG00000189320,FAM180A,protein_coding,chr7:135728348-135748813;-1,7,1,7,GGAGAGCGTCAAATGAAGAG|GGAGGCTGAAGGCTGCGTCC
AAACCAAAGAGTACGG-HEK293T_Batch1,HEK293T_Batch1,886.0,True,6636,2,20847.0,non-targeting_03016|non-targeting_03214,4.250012,None,control_nontargeting,None,None,None,1,0,GGAGGCTATCTGCTAGACTT|GTCCACATGCGGATAGACAC
AAACCAAAGAGTGGGC-HEK293T_Batch1,HEK293T_Batch1,610.0,True,5353,2,14035.0,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,4.346277,ENSG00000242419,PCDHGC4,protein_coding,chr5:141484997-141512975;1,5,1,5,GGGCCGAACCGCGCCAGAGC|GCGACCGAGAGGCACTGTGT
AAACCAAAGAGTGTAA-HEK293T_Batch1,HEK293T_Batch1,589.0,True,6358,2,20939.0,USP30_P1P2-1|USP30_P1P2-2,2.812933,ENSG00000135093,USP30,protein_coding,chr12:109023089-109088023;1,12,1,12,GTGCCGAAGAGGCCGGGACC|GACGACGGTTCCCGAGACAG
AAACCAAAGATACGGG-HEK293T_Batch1,HEK293T_Batch1,476.0,True,5527,2,14083.0,KLRD1_P1P2-1|KLRD1_P1P2-2,3.379961,ENSG00000134539,KLRD1,protein_coding,chr12:10226058-10329608;1,12,1,12,GATGTTCTTGAATCATCTAT|GGGTTCCTGGAACACTTCAG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,HEK293T_Batch223,522.0,True,8112,2,29296.0,non-targeting_00183|non-targeting_01885,1.781813,None,control_nontargeting,None,None,None,1,0,GGGGGAATCTGCAAGACCAT|GAGAAGGAGCGAGGCTAAAG
GTTGTCTTCAACAGCC-HEK293T_Batch223,HEK293T_Batch223,3035.0,True,10568,2,88551.0,AKAP13_P1P2-1|AKAP13_P1P2-2,3.427403,ENSG00000170776,AKAP13,protein_coding,chr15:85380571-85749358;1,15,1,15,GGAGCACAGGCGCTTCACCG|GGGCAGTCTCGGCAGAGCAC
GTTGTCTTCAGTACTG-HEK293T_Batch223,HEK293T_Batch223,1808.0,True,10735,2,81286.0,TTF2_P1P2-1|TTF2_P1P2-2,2.224245,ENSG00000116830,TTF2,protein_coding,chr1:117060294-117107453;1,1,1,1,GAAGTTAGGTGTCCAGAGCA|GAGGTGTCCAGAGCACGGTA


### Add metadata

In [27]:
cur_data.create_columns(
    slot="obs",
    overwrite=True,
    col_dict={
        "dataset_id": cur_data.dataset_id,
        "sample_id": range(1, cur_data.adata.obs.shape[0] + 1),
        # treatment
        "treatment_label": None,
        "treatment_id": None,
        # perturbation type
        "perturbation_type_label": "CRISPRi",
        "perturbation_type_id": None,
        # model system
        "model_system_label": "cell_line",
        "model_system_id": None,

        "data_modality": "Perturb-seq",
        "significant": None,
        "significance_criteria": None,
        "score_interpretation": None,

        # replicates
        "technical_replicate": None,
        "biological_replicate": None,

        'tissue': 'kidney',
        'cell_type': 'kidney epithelial cell',
        'cell_line': 'HEK293T cell',
        "timepoint": "P0DT0H0M0S",
        'disease_label': 'healthy',
        'disease_id':None,

        "species": "Homo sapiens",
        "sex_label": "female",
        "sex_id": None,
        "developmental_stage_label": "embryonic",
        "developmental_stage_id": None,

        "study_title": "X-Atlas/Orion: Genome-wide Perturb-seq Datasets via a Scalable Fix-Cryopreserve Platform for Training Dose-Dependent Biological Foundation Models",
        "study_uri": "https://doi.org/10.1101/2025.06.11.659105",
        "study_year": 2025,
        "first_author": "Ann Huang",
        "last_author": "Ci Chu",

        "experiment_title": "Over 4.5 million HEK293T cells were transfected with dual gRNAs targeting the same gene, and whole genome CRISPRi Perturb-seq was performed.",
        "experiment_summary": "The researchers developed the Fix-Cryopreserve-ScRNAseq (FiCS) Perturb-seq platform to enable scalable and reproducible genome-wide perturbation screening in human cell lines. Using this platform, they generated a genome-wide dual-guide CRISPRi Perturb-seq dataset in HEK293T cells. The dataset includes over 4.5 million deeply sequenced cells, with each cell yielding over 16,000 unique molecular identifiers (UMIs).",
        "number_of_perturbed_targets": len(set(cur_data.adata.obs['perturbed_target_coord'])),
        "number_of_perturbed_samples": cur_data.adata.obs.shape[0],

        "library_generation_type_id": "EFO:0022868",
        "library_generation_type_label": "endogenous",

        "library_generation_method_id": "EFO:0022895",
        "library_generation_method_label": "dCas9-KRAB",

        "enzyme_delivery_method_id": None,
        "enzyme_delivery_method_label": "nanoparticle-mediated transfection",

        "library_delivery_method_id": None,
        "library_delivery_method_label": "lentivirus transduction",

        "enzyme_integration_state_id": None,
        "enzyme_integration_state_label": "random locus integration",

        "library_integration_state_id": None,
        "library_integration_state_label": "random locus integration",

        "enzyme_expression_control_id": None,
        "enzyme_expression_control_label": "constitutive transgene expression",

        "library_expression_control_id": None,
        "library_expression_control_label": "constitutive transgene expression",

        "library_name": "custom",
        "library_uri": None,

        "library_format_id": None,
        "library_format_label": "pooled",

        "library_scope_id": None,
        "library_scope_label": "genome-wide",

        "library_perturbation_type_id": None,
        "library_perturbation_type_label": "inhibition",

        "library_manufacturer": "Weissman",
        "library_lentiviral_generation": "3",
        "library_grnas_per_target": "2",
        "library_total_grnas": "20845",
        "library_total_variants": None,

        "readout_dimensionality_id": None,
        "readout_dimensionality_label": "high-dimensional assay",

        "readout_type_id": None,
        "readout_type_label": "transcriptomic",

        "readout_technology_id": None,
        "readout_technology_label": "single-cell rna-seq",

        "method_name_id": None,
        "method_name_label": "Perturb-seq",

        "method_uri": None,

        "sequencing_library_kit_id": None,
        "sequencing_library_kit_label": "10x Genomics Chromium GEM-X Single Cell 5-prime kit v3",

        "sequencing_platform_id": None,
        "sequencing_platform_label": "Illumina NovaSeq X Plus",

        "sequencing_strategy_id": None,
        "sequencing_strategy_label": "barcode sequencing",

        "software_counts_id": None,
        "software_counts_label": "CellRanger",

        "software_analysis_id": None,
        "software_analysis_label": "custom",

        "reference_genome_id": None,
        "reference_genome_label": "GRCh38",

        "license_label": "CC BY-NC-SA 4.0",
        "license_id": "SWO:1000090",

        "associated_datasets": json.dumps([
            {
                "dataset_accession": "HEK293T_filtered_dual_guide_cells",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/55074802",
                "dataset_description": "Filtered raw counts",
                "dataset_file_name": "HEK293T_filtered_dual_guide_cells.h5ad",
            }
        ])
    }
)

Column dataset_id added to adata.obs
Column sample_id added to adata.obs
Column treatment_label added to adata.obs
Column treatment_id added to adata.obs
Column perturbation_type_label added to adata.obs
Column perturbation_type_id added to adata.obs
Column model_system_label added to adata.obs
Column model_system_id added to adata.obs
Column data_modality added to adata.obs
Column significant added to adata.obs
Column significance_criteria added to adata.obs
Column score_interpretation added to adata.obs
Column technical_replicate added to adata.obs
Column biological_replicate added to adata.obs
Column tissue added to adata.obs
Column cell_type added to adata.obs
Column cell_line added to adata.obs
Column timepoint added to adata.obs
Column disease_label added to adata.obs
Column disease_id added to adata.obs
Column species added to adata.obs
Column sex_label added to adata.obs
Column sex_id added to adata.obs
Column developmental_stage_label added to adata.obs
Column developmental_st

In [28]:
cur_data.adata.obs

,sample,total_counts_mt,pass_guide_filter,n_genes_by_counts,num_features,total_counts,perturbation_name,pct_counts_mt,perturbed_target_ensg,perturbed_target_symbol,...,sequencing_strategy_label,software_counts_id,software_counts_label,software_analysis_id,software_analysis_label,reference_genome_id,reference_genome_label,license_label,license_id,associated_datasets
index,,,,,,,,,,,,,,,,,,,,,
AAACCAAAGAAGTAGC-HEK293T_Batch1,HEK293T_Batch1,647.0,True,6856,2,24243.0,FAM180A_P1P2-1|FAM180A_P1P2-2,2.668812,ENSG00000189320,FAM180A,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
AAACCAAAGAGTACGG-HEK293T_Batch1,HEK293T_Batch1,886.0,True,6636,2,20847.0,non-targeting_03016|non-targeting_03214,4.250012,None,control_nontargeting,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
AAACCAAAGAGTGGGC-HEK293T_Batch1,HEK293T_Batch1,610.0,True,5353,2,14035.0,PCDHGC4_P1P2-1|PCDHGC4_P1P2-2,4.346277,ENSG00000242419,PCDHGC4,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
AAACCAAAGAGTGTAA-HEK293T_Batch1,HEK293T_Batch1,589.0,True,6358,2,20939.0,USP30_P1P2-1|USP30_P1P2-2,2.812933,ENSG00000135093,USP30,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
AAACCAAAGATACGGG-HEK293T_Batch1,HEK293T_Batch1,476.0,True,5527,2,14083.0,KLRD1_P1P2-1|KLRD1_P1P2-2,3.379961,ENSG00000134539,KLRD1,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTCCGTGGGATGG-HEK293T_Batch223,HEK293T_Batch223,522.0,True,8112,2,29296.0,non-targeting_00183|non-targeting_01885,1.781813,None,control_nontargeting,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
GTTGTCTTCAACAGCC-HEK293T_Batch223,HEK293T_Batch223,3035.0,True,10568,2,88551.0,AKAP13_P1P2-1|AKAP13_P1P2-2,3.427403,ENSG00000170776,AKAP13,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."
GTTGTCTTCAGTACTG-HEK293T_Batch223,HEK293T_Batch223,1808.0,True,10735,2,81286.0,TTF2_P1P2-1|TTF2_P1P2-2,2.224245,ENSG00000116830,TTF2,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HEK293T_filtered_dual_..."


### Curate replicate information

In [29]:
cur_data.adata.obs['technical_replicate'] = cur_data.adata.obs['sample'].astype(str)

### Curate tissue information

In [30]:

cur_data.standardize_ontology(
    input_column='tissue',
    column_type='term_name',
    ontology_type='tissue',
    overwrite=True
)

Mapped 1 tissue ontology terms from `tissue` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
  input_column input_column_lower name_lower     ontology_id
0       kidney             kidney     kidney  UBERON:0002113
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell type information

In [31]:
cur_data.standardize_ontology(
    input_column='cell_type',
    column_type='term_name',
    ontology_type='cell_type'
)

Mapped 1 cell_type ontology terms from `cell_type` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
             input_column      input_column_lower              name_lower  \
0  kidney epithelial cell  kidney epithelial cell  kidney epithelial cell   

  ontology_id  
0  CL:0002518  
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell line information

In [32]:
cur_data.standardize_ontology(
    input_column='cell_line',
    column_type='term_name',
    ontology_type='cell_line'
)

Mapped 1 cell_line ontology terms from `cell_line` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
   input_column input_column_lower    name_lower  ontology_id
0  HEK293T cell       hek293t cell  hek293t cell  CLO:0037372
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate disease information

In [33]:
cur_data.standardize_ontology(
    input_column='disease_label',
    column_type='term_name',
    ontology_type='disease'
)

### Match schema column order

In [34]:
cur_data.match_schema_columns(slot='obs')

Matched columns of adata.obs to the obs_schema.


### Validate obs metadata

In [36]:
sch = ObsSchema.to_schema()

for col in list(sch.columns.keys()):
    sch_subset = sch.select_columns([col])
    try:
        sch_subset.validate(cur_data.adata.obs[[col]], lazy=True)
        print(f"Validation successful for column {col}.")
    except Exception as e:
        print(f"Validation failed for column {col}: {e}")

Validation successful for column dataset_id.
Validation successful for column sample_id.
Validation successful for column data_modality.
Validation successful for column significant.
Validation successful for column significance_criteria.
Validation successful for column perturbation_name.
Validation successful for column perturbed_target_coord.
Validation successful for column perturbed_target_chromosome.
Validation successful for column perturbed_target_chromosome_encoding.
Validation successful for column perturbed_target_number.
Validation successful for column perturbed_target_ensg.
Validation successful for column perturbed_target_symbol.
Validation successful for column perturbed_target_biotype.
Validation successful for column guide_sequence.
Validation successful for column perturbation_type_label.
Validation successful for column perturbation_type_id.
Validation successful for column timepoint.
Validation successful for column treatment_label.
Validation successful for column

In [ ]:
# cur_data.validate_data(slot='obs')

# VAR slot curation

### Standardise genes

In [37]:
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 1)
--------------------------------------------------
                     gene_symbol
DDX11L2                  DDX11L2
MIR1302-2HG          MIR1302-2HG
FAM138A                  FAM138A
ENSG00000290826  ENSG00000290826
OR4F5                      OR4F5
...                          ...
ENSG00000277836  ENSG00000277836
ENSG00000278633  ENSG00000278633
ENSG00000276017  ENSG00000276017
ENSG00000278817  ENSG00000278817
ENSG00000277196  ENSG00000277196

[38606 rows x 1 columns]
--------------------------------------------------


In [38]:
cur_data.adata.var['ensembl_inputs'] = [e if e.startswith('ENSG') else None for e in cur_data.adata.var.index]
cur_data.adata.var['gene_inputs'] = [e if not e.startswith('ENSG') else '' for e in cur_data.adata.var.index]

var_df = cur_data.adata.var.copy()

In [39]:
cur_data.create_columns(slot='var', col_dict={
    'genes':cur_data.adata.var.index
})

Column genes added to adata.var


In [40]:
cur_data.standardize_genes(
    slot="var", input_column="gene_inputs", input_column_type="gene_symbol"
)
var_symb = cur_data.adata.var.copy()

Mapping gene symbols: 100%|████████████████████████████████| 25645/25645 [00:01<00:00, 23291.78it/s]


--------------------------------------------------
Successfully mapped 25586 out of 25645 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['', 'ARHGEF19-AS1', 'LYST-AS1', 'LINC01115-1', 'LINC01119', 'LSP1P5-1', 'LINC01889', 'UPP2-IT1', 'LINC01238-1', 'ARF4-AS1', 'DENND6A-AS1', 'PRICKLE2-AS1-1', 'SLC41A3-AS1', 'IFT122P3', 'LINC00886', 'LINC02263', 'TEMN3-AS1', 'CTD-2350J17.1', 'LINC02119', 'LINC01033', 'CTC-338M12.4', 'MIR5689HG', 'RAET1E-AS1-1', 'LINC03021-1', 'LINC03023-1', 'TNFRSF10A-DT-1', 'LINC01605-1', 'NCRNA00250', 'PTGER4P2-CDK2AP2P2', 'LINC03025-1', 'LINC00484-1', 'LINC02913', 'DNAJC9-AS1-1', 'LINC02722', 'LINC02398', 'GPR84-AS1-1', 'LINC02399', 'SPATA13-1', 'SMIM2', 'SMIM2-IT1', 'PCDH9-AS4', 'HIF1A-AS1', 'HIF1A-AS3', 'VASH1-AS1', 'GOLGA8M-1', 'LINC02256-1', 'LINC01580', 'DDX11L16-1', 'NPIPA9-1', 'LINC02911', 'MKKS-1', 'MKKS-2', 'D21S2088E', 'LINC01422', 'ELFN2-1', 'ARMCX5-GPRASP2-1', 'PRY2', 'PRY', 'MAFIP']
--------------------------

In [43]:
cur_data.standardize_genes(
    slot="var", input_column="ensembl_inputs", input_column_type="ensembl_gene_id"
)
var_ensg = cur_data.adata.var.copy()

Missing Ensembl IDs: ['ENSG00000237262', 'ENSG00000277630', 'ENSG00000287400', 'ENSG00000258196', 'ENSG00000232597', 'ENSG00000276075', 'ENSG00000258918', 'ENSG00000234810', 'ENSG00000263081', 'ENSG00000289919', 'ENSG00000286542', 'ENSG00000254836', 'ENSG00000229458', 'ENSG00000291259', 'ENSG00000254420', 'ENSG00000253988', 'ENSG00000253369', 'ENSG00000286397', 'ENSG00000266588', 'ENSG00000287824', 'ENSG00000257879', 'ENSG00000247810', 'ENSG00000290010', 'ENSG00000290591', 'ENSG00000240996', 'ENSG00000290066', 'ENSG00000272755', 'ENSG00000228421', 'ENSG00000289209', 'ENSG00000272732', 'ENSG00000288991', 'ENSG00000286889', 'ENSG00000290510', 'ENSG00000276250', 'ENSG00000289620', 'ENSG00000289036', 'ENSG00000286931', 'ENSG00000289467', 'ENSG00000273771', 'ENSG00000284930', 'ENSG00000276760', 'ENSG00000235209', 'ENSG00000240355', 'ENSG00000286895', 'ENSG00000287798', 'ENSG00000258844', 'ENSG00000131484', 'ENSG00000236856', 'ENSG00000288963', 'ENSG00000264080', 'ENSG00000286778', 'ENSG0000

# Compile final gene symbols and ensg ids from the two mappings


In [44]:
var_df_clean = var_symb.merge(var_ensg, left_index=True, right_index=True, suffixes=('_symb', '_ensg'))
var_df_clean = var_df_clean.drop(columns=['gene_inputs_symb', 'gene_inputs_ensg', 'ensembl_inputs_symb', 'ensembl_inputs_ensg'])
var_df_clean['clean_gene_symbol'] = None
var_df_clean['clean_ensg_id'] = None

ensg_starts_mask = var_df_clean.index.str.startswith('ENSG')

# if original id starts with ENSG - assign ensg mapping
var_df_clean.loc[ensg_starts_mask, 'clean_gene_symbol'] = var_df_clean.loc[ensg_starts_mask, 'gene_symbol_ensg']
var_df_clean.loc[ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[ensg_starts_mask, 'ensembl_gene_id_ensg']

# if not, assign symb mapping
var_df_clean.loc[~ensg_starts_mask, 'clean_gene_symbol'] = var_df_clean.loc[~ensg_starts_mask, 'gene_symbol_symb']
var_df_clean.loc[~ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[~ensg_starts_mask, 'ensembl_gene_id_symb']

# replace missing ensg with
var_df_clean.loc[var_df_clean['clean_ensg_id'].isna() & ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[var_df_clean['clean_ensg_id'].isna() & ensg_starts_mask].index

var_df_clean = var_df_clean.rename(columns={'clean_gene_symbol':'gene_symbol', 'clean_ensg_id':'ensembl_gene_id'})[['ensembl_gene_id', 'gene_symbol']]

var_df_clean


,ensembl_gene_id,gene_symbol
index,,
DDX11L2,ENSG00000310539,DDX11L2
MIR1302-2HG,ENSG00000243485,MIR1302-2HG
FAM138A,ENSG00000237613,FAM138A
ENSG00000290826,ENSG00000290826,None
OR4F5,ENSG00000186092,OR4F5
...,...,...
ENSG00000277836,ENSG00000277836,NaN
ENSG00000278633,ENSG00000278633,NaN
ENSG00000276017,ENSG00000276017,NaN


In [45]:
# assign fixed genes back to var
cur_data.adata.var = var_df_clean

In [46]:
var_df_clean.loc[var_df_clean['ensembl_gene_id'].notna() & var_df_clean['ensembl_gene_id'].str.startswith('LRG')]

,ensembl_gene_id,gene_symbol
index,,


### Validate var metadata

In [49]:
cur_data.validate_data(slot='var')

,ensembl_gene_id,gene_symbol
index,,
DDX11L2,ENSG00000310539,DDX11L2
MIR1302-2HG,ENSG00000243485,MIR1302-2HG
FAM138A,ENSG00000237613,FAM138A
ENSG00000290826,ENSG00000290826,None
OR4F5,ENSG00000186092,OR4F5
...,...,...
ENSG00000277836,ENSG00000277836,NaN
ENSG00000278633,ENSG00000278633,NaN
ENSG00000276017,ENSG00000276017,NaN


# Save the dataset

In [50]:
cur_data.save_curated_data_h5ad()

/content/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)


✅ Curated h5ad data saved to Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad


In [51]:
cur_data.save_curated_data_parquet(split_metadata=True, save_metadata_only=True)

✅ Metadata saved to Perturbseq/curated/parquet/orion_2025_hek293t_curated_metadata.parquet


# Upload to BigQuery

In [52]:
upload_parquet_to_bq(
    parquet_path='/content/PerturbationCatalogue/data_exploration/Perturbseq/curated/parquet/orion_2025_hek293t_curated_metadata.parquet',
    bq_dataset_id='prj-ext-dev-pertcat-437314.perturb_seq',
    bq_table_name='metadata',
    key_columns=['dataset_id', 'sample_id'],
    verbose=True
)

Staging table: loading `.parquet` file /content/PerturbationCatalogue/data_exploration/Perturbseq/curated/parquet/orion_2025_hek293t_curated_metadata.parquet to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging...
Staging table: loaded 4534299 rows to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging
Staging table: added ingested_at timestamp column to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging
Merge completed: staging → prj-ext-dev-pertcat-437314.perturb_seq.metadata with type-safe casting.
Staging table: deleted prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging


# Upload to GC Storage

In [53]:
!gcloud storage cp Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

uploading large objects. If you would like to opt-out and instead
perform a normal upload, run:
`gcloud config set storage/parallel_composite_upload_enabled False`
If you would like to disable this warning, run:
`gcloud config set storage/parallel_composite_upload_enabled True`
Note that with parallel composite uploads, your object might be
uploaded as a composite object
(https://cloud.google.com/storage/docs/composite-objects), which means
that any user who downloads your object will need to use crc32c
checksums to verify data integrity. gcloud storage is capable of
computing crc32c checksums, but this might pose a problem for other
clients.

Copying file://Perturbseq/curated/h5ad/orion_2025_hek293t_curated.h5ad to gs://perturbation-catalogue-lake/perturbseq/curated/orion_2025_hek293t_curated.h5ad

Average throughput: 354.8MiB/s
